In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("company.db")
cursor = conn.cursor()

# Create Employees table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Department TEXT NOT NULL,
        Salary INTEGER NOT NULL,
        Hire_Date TEXT NOT NULL
    )
''')

# Create Departments table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL UNIQUE,
        Manager TEXT NOT NULL
    )
''')

# Insert sample data into Employees
cursor.executemany('''
    INSERT INTO Employees (Name, Department, Salary, Hire_Date) VALUES (?, ?, ?, ?)
''', [
    ("Alice", "Sales", 50000, "2021-01-15"),
    ("Bob", "Engineering", 70000, "2020-06-10"),
    ("Charlie", "Marketing", 60000, "2022-03-20")
])

# Insert sample data into Departments
cursor.executemany('''
    INSERT INTO Departments (Name, Manager) VALUES (?, ?)
''', [
    ("Sales", "Alice"),
    ("Engineering", "Bob"),
    ("Marketing", "Charlie")
])

# Commit and close connection
conn.commit()
conn.close()

In [5]:
pip install flask nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
from flask import Flask, request, jsonify
import sqlite3
import re
import nltk

nltk.download('punkt')

app = Flask(__name__)

# Function to connect to the SQLite database
def connect_db():
    return sqlite3.connect("company.db")

# Function to process user queries
def process_query(user_input):
    user_input = user_input.lower()

    # Match queries using regular expressions
    if re.search(r"employees in (\w+)", user_input):
        dept = re.search(r"employees in (\w+)", user_input).group(1)
        return f"SELECT Name, Salary, Hire_Date FROM Employees WHERE Department='{dept}';"
    
    elif re.search(r"manager of (\w+)", user_input):
        dept = re.search(r"manager of (\w+)", user_input).group(1)
        return f"SELECT Manager FROM Departments WHERE Name='{dept}';"

    elif re.search(r"hired after (\d{4}-\d{2}-\d{2})", user_input):
        date = re.search(r"(\d{4}-\d{2}-\d{2})", user_input).group(1)
        return f"SELECT Name, Hire_Date FROM Employees WHERE Hire_Date > '{date}';"

    elif re.search(r"salary expense for (\w+)", user_input):
        dept = re.search(r"salary expense for (\w+)", user_input).group(1)
        return f"SELECT SUM(Salary) FROM Employees WHERE Department='{dept}';"
    
    else:
        return None  # If no valid query is found

# API Endpoint
@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.json.get("query")
    
    if not user_input:
        return jsonify({"error": "No query provided"}), 400

    sql_query = process_query(user_input)

    if sql_query:
        conn = connect_db()
        cursor = conn.cursor()

        try:
            cursor.execute(sql_query)
            result = cursor.fetchall()
            conn.close()
            
            if not result:
                return jsonify({"response": "No results found."})
            
            return jsonify({"response": result})
        
        except Exception as e:
            return jsonify({"error": str(e)}), 500

    return jsonify({"error": "Sorry, I didn't understand that query."}), 400

if __name__ == "__main__":
    app.run(debug=True)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Lenovo\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
